In [2]:
import os,sys
import glob
import time,datetime
import numpy as np
import pandas as pd
from operator import itemgetter

In [3]:
%run ../tools/TrainTest.py
%run ../tools/PCG1.py

In [4]:
# set dataset
groundtruth_csv = '../dataset/open_peer_review/peer_review/translated_groundtruth.csv'
peer_review_csv = '../dataset/open_peer_review/peer_review/peer_review_forPG3_suffled.csv'
gDF = pd.read_csv(groundtruth_csv)
rDF = pd.read_csv(peer_review_csv)

### PCG1

In [3]:
# path information
train_fold_num = 5
snapshot_path = '../result/snapshot/ability_estimation/'
snapshot_form = 'PCG1-hyper6-EI5000w500-fold{0}-*'

In [21]:
for i in xrange(train_fold_num):
    fold_id = i
    # preprocess train&test dataset
    rBlocks = foldDataset(rDF, train_fold_num)
    trainDF = rBlocks.pop(fold_id).reset_index(drop=True)
    testDF = pd.concat(rBlocks).reset_index(drop=True)

    fname = glob.glob(os.path.join(snapshot_path,snapshot_form.format(fold_id)))[0]
    print(fname)
    npz = np.load(fname)
    X = npz['arr_0']
    Y = npz['arr_1']
    optX = X[Y.argmin()]
    optY = Y.min()
    print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))
    
    # testing
    testModel = PCG1(gDF,testDF)
    cor = testModel.corrcoefWithTruth(optX)
    print('Test correlation coefficient: {}'.format(cor))

../result/snapshot/ability_estimation/PCG1-hyper6-EI5000w500-fold0-20170802055927.npz
opt_x: [  1.36963758   4.97572434  90.49518224  58.56319179   2.13512864
  77.60876654], opt_fx: -0.321585474961
Test correlation coefficient: 0.42125995605
../result/snapshot/ability_estimation/PCG1-hyper6-EI5000w500-fold1-20170901214005.npz
opt_x: [  1.97673118   4.40754458  97.66956408  80.13444856  81.37397913
  92.81235314], opt_fx: -0.316572147581
Test correlation coefficient: 0.411173067617
../result/snapshot/ability_estimation/PCG1-hyper6-EI5000w500-fold2-20170902030535.npz
opt_x: [  2.3174495    6.77544424  90.93826995  54.19241159   6.666481
  11.71925763], opt_fx: -0.281436583471
Test correlation coefficient: 0.430618077519
../result/snapshot/ability_estimation/PCG1-hyper6-EI5000w500-fold3-20170902101102.npz
opt_x: [  2.52437065   2.35236233  59.62614016  68.54550457   5.4254085
  51.91273995], opt_fx: -0.32346203171
Test correlation coefficient: 0.434660264894
../result/snapshot/ability_es

#### unopen_data

In [5]:
snapshot1 = '../result/snapshot/unopen_data/PCG1-hyper6-EI5000w500-20170708151847.npz'
snapshot2 = '../result/snapshot/unopen_data/PCG1-hyper6-EI5000w500-20170710060217.npz'

In [6]:
npz = np.load(snapshot1)
X = npz['arr_0']
Y = npz['arr_1']
optX = X[Y.argmin()]
optY = Y.min()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [  0.62605843   2.71787038  77.04591334  81.12385285  20.97974444
  49.67641729], opt_fx: -0.455243044715


In [7]:
npz = np.load(snapshot2)
X = npz['arr_0']
Y = npz['arr_1']
optX = X[Y.argmin()]
optY = Y.min()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [  2.54785885   4.62864443  67.68716059  22.95702442   6.96275866
  56.26711153], opt_fx: -0.465397822629


#### v2 full

In [29]:
snapshotFull = '../result/snapshot/ability_estimation_bo/PCG1-hyper6-RAM5000w500-full-20171113143358.npz'

In [36]:
npz = np.load(snapshotFull)
mu,gamma,alpha,bata,eta,kappa = npz['mu0'],npz['gamma0'],npz['alpha0'],npz['beta0'],npz['eta0'],npz['kappa0']
Y = npz['target']
optX = [mu[Y.argmax()],gamma[Y.argmax()],alpha[Y.argmax()],bata[Y.argmax()],eta[Y.argmax()],kappa[Y.argmax()]]
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [1.2888962120845395, 3.8099811639564529, 88.301983181302546, 65.366092662031335, 1.6036397981255406, 14.096788739485559], opt_fx: 0.456591964029


In [32]:
npz.files

['alpha0', 'eta0', 'mu0', 'gamma0', 'kappa0', 'beta0', 'target']

In [7]:
for i in xrange(len(Y)):
    print('{0}:{1}'.format(X[i],Y[i]))

[  3.91399423  23.47636587  70.50896851  17.95693007  20.09796207
  78.8562371 ]:[-0.04647122]
[  2.16851706  48.83637708  62.70963086  30.30621605  10.09667918
  84.59934826]:[-0.22536716]
[  1.28312054  99.67014114   0.41646583  83.09316768  16.41157844
  66.83360753]:[-0.15927528]
[  3.23059323  56.65335174  90.27940173  71.3378158   60.93765093
  96.75178413]:[-0.08213982]
[  2.49436092  45.1961402   59.29801225  36.39373829  80.01861922
  28.06953214]:[-0.23018504]
[  1.08344602  38.02923548  58.48621878  51.88179452  84.62005631
  71.96668846]:[-0.12928589]
[  3.45856939  45.19313737  43.58000667  93.3207684   11.04429299
  17.48076981]:[-0.13949723]
[  3.80004621  47.21794942  90.25732846  22.6722983   74.25650699
  24.41883456]:[-0.01358219]
[  2.44526974  72.73383451  33.11694219  55.11825137  50.40280094
  30.64939126]:[-0.21013859]
[  3.71489616  46.80257901  57.27901541  28.09505389  91.11706541
  37.0537677 ]:[-0.02028464]
[  2.3062238   14.66940195  87.63200475   4.676059

### PCG4
#### open_v2

In [7]:
snapshot1 = '../result/snapshot/ability_estimation_bo/PCG4-hyper5-RAM5000w500-full-20171116122326.npz'

In [8]:
npz = np.load(snapshot1)
mu,beta,gamma,eta,kappa = npz['mu0'],npz['beta0'],npz['gamma0'],npz['eta0'],npz['kappa0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],beta[a],eta[a],kappa[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [2.3277801283939112, 1.7688387713219633, 84.058557180233933, 2.0194772787841155, 22.503846951866464], opt_fx: 0.514016338976


In [9]:
for i in xrange(len(Y)):
    print(i,Y[i],X(i))

(0, 0.48010569875911213, [0.71697778240817511, 0.72579738998349286, 30.293774501149905, 22.451562690211759, 54.481864347079402])
(1, 0.39595153582132797, [2.5424087657784633, 8.7154538981353049, 57.668006337856475, 17.446774626292534, 79.5106655507754])
(2, 0.42252646094149299, [3.0536289927186706, 3.2615588827651507, 55.123870091223949, 9.445722406707592, 22.865980835939432])
(3, 0.036694631922767999, [3.9292281379504304, 7.7892612868938684, 98.532135374682625, 97.935898061185227, 2.2925765786434487])
(4, 0.3549926503435622, [2.6382336357942129, 6.5870930743197489, 40.072700804846072, 84.89705669539758, 5.5162708919553847])
(5, 0.49386939026802174, [1.5130462449503463, 3.6211167339399277, 52.800858821101905, 5.5361836387175343, 81.170238082630675])
(6, 0.48549253630648376, [1.9093841036343062, 1.2061159560460724, 21.33837257211589, 46.922381943127085, 86.260825777122321])
(7, 0.40065512232479955, [2.1307277840804932, 9.5554986269805031, 70.47075789417282, 33.195735443589314, 48.073020

### PCG5

#### open_v2

In [1]:
snapshot1 = '../result/snapshot/ability_estimation_bo/PCG5-hyper6-RAM5000w500-full-20171120192041.npz'

In [5]:
npz = np.load(snapshot1)
mu,beta,gamma,eta,lambda0,kappa = npz['mu0'],npz['beta0'],npz['gamma0'],npz['eta0'],npz['lambda0'],npz['kappa0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],beta[a],eta[a],lambda0[a],kappa[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [2.0482204711158492, 2.0238687592430828, 66.418948054680456, 61.528112510945839, 26.35739405848437, 4.1202387943164869], opt_fx: 0.523510597541


### PG1
#### open_v2

In [4]:
snapshot1 = '../result/snapshot/ability_estimation_bo/PG1-hyper5-RAM5000w500-full-20171114040231.npz'

In [5]:
npz = np.load(snapshot1)
mu,alpha,beta,gamma,eta = npz['mu0'],npz['alpha0'],npz['beta0'],npz['gamma0'],npz['eta0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],alpha[a],beta[a],eta[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [1.5470505925340223, 2.6435065376167355, 62.677781879467531, 78.700057185562258, 1.3352937328431147], opt_fx: 0.455020027369


In [6]:
for i in xrange(len(Y)):
    print(i,Y[i],X(i))

(0, 0.36343143942926437, [0.17525121247106679, 2.5800906760565412, 43.939746008881919, 30.593981462482855, 4.5291070273541632])
(1, 0.43266889584986079, [0.71055439441038537, 3.8749920578925172, 32.210942194657655, 47.455409476409081, 1.7257392951148156])
(2, 0.41576341079322804, [2.4307505378541276, 4.7004579427368043, 29.864912905024049, 97.827413157562063, 1.8456133043473151])
(3, 0.41541861973680227, [1.1291070859081516, 2.6288175373370306, 92.608367548792145, 41.169323938946576, 2.8809681098323288])
(4, 0.34106523080634588, [0.42387546754246053, 1.5349749694506101, 47.796788684402017, 33.709639635009474, 4.9589637858202789])
(5, 0.41728911295307913, [1.81381748310256, 2.4742157372796156, 59.525418951537837, 24.142840356899793, 4.5583458516374096])
(6, 0.44858018938877581, [1.9314460016945287, 3.5729953278270954, 14.565996254124354, 30.310596455002166, 2.5456343195265729])
(7, 0.43137289243269261, [2.7099315787503011, 4.9205635705357054, 30.931347105066575, 3.1999168585449187, 4.35

### PG3
#### open_v1

In [16]:
snapshot1 = '../result/snapshot/ability_estimation/PG3-hyper5-EI5000w500-fold0-20170801233134.npz'
snapshot2 = '../result/snapshot/ability_estimation/PG3-hyper5-EI5000w500-fold1-20170831180817.npz'
snapshot3 = '../result/snapshot/ability_estimation/PG3-hyper5-EI5000w500-fold2-20170901000656.npz'

In [26]:
npz = np.load(snapshot2)
X = npz['arr_0']
Y = npz['arr_1']
optX = X[Y.argmin()]
optY = Y.min()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [  1.72067923  22.21440316  10.52636159   0.45665194  22.17691917], opt_fx: -0.341719366835


In [27]:
for i in xrange(len(Y)):
    print(i,Y[i],X[i])

(0, array([ 0.01981445]), array([  1.69646418,   1.71028138,  95.55951903,  51.85003464,  17.70291529]))
(1, array([ 0.24153521]), array([  2.39702069,  84.60230007,  21.33929129,  50.27788559,  74.61965843]))
(2, array([ 0.02221655]), array([  3.20416258,  32.55852097,  18.7798627 ,  19.68076608,  60.30853315]))
(3, array([-0.05909228]), array([  3.8547186 ,  63.68423321,  60.74801319,  29.97711419,  96.76926502]))
(4, array([-0.28949602]), array([  1.0580612 ,  49.96018768,  56.90685986,   1.3980176 ,  80.70375339]))
(5, array([ 0.01376462]), array([  1.1118431 ,  25.21583049,  85.27564301,  70.17480855,  30.45132305]))
(6, array([ 0.213176]), array([  2.19861424,  11.2997339 ,   7.11991887,  98.08672368,   0.12107049]))
(7, array([ 0.20194807]), array([  0.79709924,  34.22747954,  16.16708109,  88.62944201,  62.85118531]))
(8, array([ 0.25820606]), array([  2.18351456,  30.05483608,  13.6006981 ,  90.27459685,   5.85076545]))
(9, array([-0.08471583]), array([  1.58458043,  84.011709

### PG4
#### open_v1

In [3]:
snapshot1 = '../result/snapshot/ability_estimation_bo/PG4-hyper4-RAM5000w500-full-20171114161140.npz'

In [6]:
npz = np.load(snapshot1)
mu,beta,gamma,eta = npz['mu0'],npz['beta0'],npz['gamma0'],npz['eta0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],beta[a],eta[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [0.28192146943278429, 2.3629077221598864, 48.879426755783996, 1.5484561530008847], opt_fx: 0.507167895822


In [7]:
for i in xrange(len(Y)):
    print(i,Y[i],X(i))

(0, 0.46515105807702289, [2.4774635449280709, 4.08635379713518, 24.432874144782609, 52.702211735919676])
(1, 0.45359130527038943, [0.47364879338113086, 0.76725017575177101, 22.578320295899115, 40.397095140042431])
(2, 0.28341502113163597, [0.4985955329260392, 4.6869916362999877, 97.404447101453584, 24.808916011927796])
(3, 0.32627276460621724, [0.96108816484043125, 3.8654788462845064, 24.187005038911071, 73.997026861226544])
(4, 0.34770399150292786, [3.2401528044109953, 3.5230234854381695, 66.372003470450423, 95.191847556527762])
(5, 0.44564605346371094, [1.2655786491964647, 1.446021485667943, 22.772029897793072, 55.949421571353341])
(6, 0.45741291105999843, [1.9853481308995606, 2.4953632816378661, 8.1239417594105738, 31.715814750496367])
(7, 0.40471009832705884, [0.88826743428365651, 4.5675324931034318, 27.188032103455008, 8.557881818925285])
(8, 0.28202934518385381, [0.51104095225315049, 4.0567566914695066, 99.050981434550764, 65.038061164965612])
(9, 0.49886188317472779, [2.01320284

### PG5
#### open_v2

In [3]:
snapshot1 = '../result/snapshot/ability_estimation/PG5-hyper5-EI5000w500-full-20171110223849.npz'
snapshot2 = '../result/snapshot/ability_estimation/PG5-hyper5-EI5000w500-fold0-20170802032825.npz'
snapshot3 = '../result/snapshot/ability_estimation_bo/PG5-hyper5-RAM5000w500-full-20171115234802.npz'

In [4]:
npz = np.load(snapshot3)
mu,beta,gamma,eta,lambda0 = npz['mu0'],npz['beta0'],npz['gamma0'],npz['eta0'],npz['lambda0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],beta[a],eta[a],lambda0[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [2.0017450149504099, 1.1083137908348422, 74.935112226275436, 26.119732943279654, 7.5011416500579973], opt_fx: 0.533313315234


In [5]:
for i in xrange(len(Y)):
    print(i,Y[i],X(i))

(0, 0.36960801799483056, [0.91705768750043326, 4.9357629066564543, 20.039200117396632, 88.251431838897048, 90.315007829140797])
(1, 0.35833023766815875, [3.2526497491051094, 7.6664953446330317, 93.459193239045575, 84.335949246541162, 56.942356175969252])
(2, 0.37188682904056336, [0.080170257571222425, 7.1894976999597091, 13.434809089686464, 71.360514836587072, 31.379480917892689])
(3, 0.36928280812294279, [1.8798389920190521, 4.9913609330815554, 72.69451538541891, 60.689582165135086, 33.170924966870516])
(4, 0.2979778411096789, [3.9175530924472799, 6.8370384525327728, 41.514204993351036, 68.882877310427915, 11.457612971394669])
(5, 0.39157978122054671, [1.8294851892237549, 3.7488557034158263, 47.159557945081396, 22.02315915189941, 62.585688225070363])
(6, 0.42426961580476402, [2.2633149207554601, 2.128825996224295, 38.390476331363651, 81.571450898820544, 31.956125220652964])
(7, 0.40013838278184771, [3.800867388774205, 6.0997546219530951, 85.137460943509225, 57.94867698102415, 79.45918

In [7]:
X[11]

array([  1.52505205e+00,   8.70856606e+01,   8.66190250e+01,
         7.66495715e+01,   4.04121947e-02])

In [ ]:
X[]

### PC1

#### open_v2

In [10]:
snapshot1 = '../result/snapshot/ability_estimation_bo/PC1-hyper4-RAM5000w500-full-20171114024802.npz'

In [11]:
npz = np.load(snapshot1)
mu,gamma,eta,kappa = npz['mu0'],npz['gamma0'],npz['eta0'],npz['kappa0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],eta[a],kappa[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [-1.8303344278867275, 2.7455660370329711, 65.261222100047618, 74.745097131645352], opt_fx: 0.207036303541


In [12]:
for i in xrange(len(Y)):
    print(i,Y[i],X(i))

(0, 0.17865769907453627, [1.1286272396655463, 7.5201576097986882, 5.5412445641488421, 43.116762416766086])
(1, 0.17101806676817863, [-0.81208004028817404, 1.0442880151179397, 61.574283044323749, 39.543646889326062])
(2, 0.18797185986970544, [-0.55869195506172753, 1.7847573765587099, 37.735258090877004, 88.932990535666548])
(3, 0.16665743873454086, [-0.77366001035270138, 9.0035743319662291, 14.902606709380638, 1.5524381248699648])
(4, 0.16280680179293161, [-1.5193118145400879, 9.2378352728235313, 31.317896297810776, 44.200631627518625])
(5, 0.15695160876779626, [1.6077309718524848, 1.527060357640516, 73.75641593958656, 7.749256714726477])
(6, 0.18028405236789549, [1.2926225076952034, 5.5252286032254947, 7.3460112913316591, 73.888921585456657])
(7, 0.16905946520116993, [1.392064576789195, 5.7159581926146723, 10.80652605898111, 72.089277597064509])
(8, 0.11486867559580025, [1.6556959555561148, 8.1546864614604608, 46.118544617116498, 74.12835791453773])
(9, 0.18667227939383121, [0.12983853

### PG4PC2

In [6]:
snapshot1 = '../result/snapshot/ability_estimation_bo/PG4PC2-hyper5-RAM5000w500-full-20171116070622.npz'

In [7]:
npz = np.load(snapshot1)
mu,beta,gamma,eta,kappa = npz['mu0'],npz['beta0'],npz['gamma0'],npz['eta0'],npz['kappa0']
Y = npz['target']
X = lambda a :[mu[a],gamma[a],beta[a],eta[a],kappa[a]]
optX = X(Y.argmax())
optY = Y.max()
print('opt_x: {0}, opt_fx: {1}'.format(optX,optY))

opt_x: [0.053282366837900774, 15.7254718551679, 31.865171622890905, 2.3239757012215394, 37.201694569165539], opt_fx: 0.355546870975


In [9]:
ls = list()
for i in xrange(len(Y)):
    ls.append([i,Y[i],X(i)])
ls.sort(key=itemgetter(1))

In [10]:
ls

[[127,
  0.031209825958899484,
  [0.69545165172972823,
   1.5168957708920499,
   9.4803373250586613,
   68.02219151242825,
   84.598680068204061]],
 [74,
  0.035141256861484546,
  [0.023381917573655819,
   1.2966928000364406,
   58.589045203674786,
   94.616337014476187,
   18.918317957118973]],
 [19,
  0.036522098899584833,
  [3.9745041524861549,
   29.851698613110685,
   86.205454504165147,
   78.59921614854747,
   52.624012317929299]],
 [116,
  0.041404780794415209,
  [3.8570955340642548,
   29.894941123439718,
   63.300098514122887,
   89.738091798340193,
   37.456513490977365]],
 [64,
  0.043405141932610296,
  [3.882048281395293,
   23.261361702016529,
   97.994352792289831,
   94.100915420227039,
   36.400269440011463]],
 [93,
  0.056743010700595904,
  [3.9387636733311164,
   18.615698835975806,
   20.890101608860725,
   63.315660139958553,
   85.169208957290124]],
 [62,
  0.070575891073668212,
  [3.6536363318771232,
   22.812786076368265,
   84.461197860896462,
   62.67648087971